In [1]:
# Code to get the data set for the OBIS workshop

In [2]:
# step 1 URL for Santa Cruz
urlapi='https://ifcb.caloos.org/api/'
url='https://ifcb.caloos.org/santa-cruz-municipal-wharf/'

In [3]:
# should create code to get data based upon feed of URL and site
import xarray as xr
import numpy as np
import pandas as pd
import ifcb_api_access

In [4]:
# Lets get some bin_id's
bin_ids=ifcb_api_access.get_bins_in_range("2023-07-17","2023-07-18")
#bin_ids=ifcb_api_access.get_bins_in_range("2022-07-17","2022-07-18")
print(bin_ids[0:3]) # print limited number of names for an idea

0    uz-municipal-wharf/D20230717T000942_IFCB104
1    uz-municipal-wharf/D20230717T003329_IFCB104
2    uz-municipal-wharf/D20230717T005715_IFCB104
Name: pid, dtype: object


In [5]:
# get the meta data for a bin ID 
# Note need to remove all the values prior to "D"
# This is not generic but works for now
bin_id=bin_ids[0][19:]
metadatavals=ifcb_api_access.get_ifcb_metadata(bin_ids[0][19:])

In [6]:
len(bin_ids)

58

In [7]:
# Do we have autoclass data for this file
#ib=np.arange(0,len(bin_ids))
ib=np.arange(0,2)
for i in ib:
    bin_id=bin_ids[i][19:]
    test=ifcb_api_access.bin_has_autoclass(bin_id)
    print(test)

True
True


In [8]:
# Not all "bins" have autoclass scores
# load Patrick's names 
class_names_file='c:/users/flbahr/class_names_matched_to_WoRMS_cencoos_classifier_20240412.csv'
class_names=pd.read_csv(class_names_file)

In [9]:
target_labels=class_names['Class Name']

In [10]:
bin_id

'D20230717T003329_IFCB104'

In [11]:
# get the api autoclass file for the data
bin_url=url+bin_id+'_class_scores.csv' # defined above
autoclass=pd.read_csv(bin_url)

In [12]:
autoclass.head(3)

,pid,Akashiwo,Alexandrium_singlet,Amy_Gony_Protoc,Asterionellopsis,Boreadinium,Centric,Ceratium,Chaetoceros,Ciliates,...,Scrip_Het,Skeletonema,Thalassionema,Thalassiosira,Tiarina,Tintinnid,Tontonia,Torodinium,Tropidoneis,Vicicitus
0,D20230717T003329_IFCB104_00002,2.400000e-07,0.000000,0.000000,6.000000e-08,6.000000e-08,7.000000e-07,0.000006,0.000003,0.000000,...,6.000000e-08,6.000000e-08,0.0,0.00000,0.0,1.000000e-07,0.000000,0.000000,6.000000e-08,0.000000e+00
1,D20230717T003329_IFCB104_00003,6.400000e-06,0.000395,0.000005,8.000000e-07,5.000000e-07,3.754000e-02,0.000001,0.000116,0.010056,...,1.842500e-03,2.400000e-07,0.0,0.00544,0.0,0.000000e+00,0.000013,0.000169,0.000000e+00,5.400000e-07
2,D20230717T003329_IFCB104_00004,1.135000e-03,0.001166,0.011470,4.300000e-06,2.300000e-06,4.343000e-01,0.005390,0.022220,0.005154,...,4.856000e-03,1.530000e-05,0.0,0.00794,0.0,1.071000e-04,0.000529,0.013840,0.000000e+00,3.000000e-07


In [13]:
# get detail file information
details=ifcb_api_access.get_bin_details(bin_id)

In [14]:
# need to drop the ml from the value below
voltmp=details['ml_analyzed'].split()
analyzed_volume=float(voltmp[0])
# we now have the volume sampled as a float so we can use it in equations

In [15]:
# Now have metadata values and details
# metadatavals and details
# autoclass for the bin
# bin_id="D20230717T003329_IFCB104"

In [16]:
# get the features file
feature_filename=bin_id+'_features.csv'
features=pd.read_csv(url+feature_filename)

In [17]:
# Show what the file contains with its header
features.head(3)

,roi_number,Area,B180,B90,Bflip,Biovolume,BoundingBox_xwidth,BoundingBox_ywidth,ConvexArea,ConvexPerimeter,...,HOG79,HOG80,HOG81,Area_over_PerimeterSquared,Area_over_Perimeter,H90_over_Hflip,H90_over_H180,Hflip_over_H180,summedConvexPerimeter_over_Perimeter,rotated_BoundingBox_solidity
0,2,54574.0,0.368497,0.181079,0.962857,3.758912e+06,575,224,90723.669102,1302.338204,...,0.308797,0.361135,0.316723,0.022128,34.750909,35.205691,2.031675,0.057709,0.829286,0.427089
1,3,1267.0,0.918046,0.825847,0.930654,3.857876e+04,48,36,1327.957732,129.915465,...,0.164193,0.264625,0.485808,0.067791,9.267747,2.203961,2.194031,0.995494,0.950295,0.699945
2,4,1912.0,0.880233,0.866455,0.888182,6.354290e+04,55,52,2247.137266,170.274532,...,0.169124,0.354672,0.445574,0.053102,10.076294,1.283148,1.064763,0.829806,0.897352,0.647341


In [18]:
#xf=features['roi_number']==33
#features[xf]

In [19]:
# okay now we have most of the pieces 
# start to construct the tables?
# don't know if the random bin id I picked has the species we are looking for above the threshold

In [20]:
# need to load the class threshold file also
# another file from Patrick that has his threshold values
threshold=pd.read_csv('c:/users/flbahr/class_thresholds_cencoos_classifier_20240412.csv')

In [21]:
# files that are unique to the deployments are the class name file and the threshold file
# these can NOT be extracted from the api as far as we know

In [22]:
threshold.head(3)

,Class Name,Threshold
0,Akashiwo,0.95
1,Alexandrium_singlet,0.49
2,Amy_Gony_Protoc,0.35


In [23]:
class_names.head(3)

,Class Name,Class ID,Common Name,AphiaID,URN,Grouping,HAB,Description,Manual Classifier Notes,Example Images
0,Akashiwo,0,Akashiwo sanguinea,232546.0,urn:lsid:marinespecies.org:taxname:232546,Dinoflagellate,Y,"Monophyletic, marine dinoflagellate","large single cells, kidney-shaped when viewed ...",NaN
1,Alexandrium_singlet,1,Alexandium sp.,109470.0,urn:lsid:marinespecies.org:taxname:109470,Dinoflagellate,Y,A. catenella and others,"Originally class was broken out for singles, d...",http://akashiwo.oceandatacenter.ucsc.edu:8000/...
2,Amy_Gony_Protoc,2,"Amylax, Gonyaulax or Protoceratium",109428.0,urn:lsid:marinespecies.org:taxname:109428,Dinoflagellate,Y,Descision was made not to distinguish between ...,“Feet” and points visible.,NaN


In [24]:
# okay we 
[nr,nc]=class_names.shape

In [25]:
import pyworms
# run through AphiaID to get scientific name, rank, kingdom
nlooks=np.arange(0,nr)
for pi in nlooks:
    # get Patrick's class name associated with that AphiaID so we can add it back into the 
    # data frame for later when we need to match things up based upon the class name he used
    cnid=class_names['Class Name'][pi]
    # Set up to catch cases that are not valid AphiaIDs (i.e. NaN is used for unclassified)
    try:
        myaid=int(class_names['AphiaID'][pi])
        wmentry=pyworms.aphiaRecordByAphiaID(myaid)
        mywp=pd.DataFrame([wmentry])
        bool=mywp['status']=='accepted'
        # if it has accepted status then create a dataframe to add to a larger dataframe of all values
        if bool[0]:
            newdf=mywp[["AphiaID","scientificname","lsid","rank","kingdom"]]
            [n,m]=newdf.shape
            # check to see if we get more than one value (probably shouldn't but I'm not sure)
            # note this fails if the first AphiaID is NaN
            if pi==0:
                # add Patrick's class name to the end
                newdf.insert(5,"Class Name",cnid,True)
                wormsdf=newdf
            else:
                newdf.insert(5,"Class Name",cnid,True)
                wormsdf=pd.concat([wormsdf,newdf])
        else:
            print("The record found is not \"accepted\"")
    except:
        print("Found NaN at "+str(pi))

Found NaN at 51


In [26]:
# display dataframe of worms information base upon Patrick's file
wormsdf.head(3)

,AphiaID,scientificname,lsid,rank,kingdom,Class Name
0,232546,Akashiwo sanguinea,urn:lsid:marinespecies.org:taxname:232546,Species,Chromista,Akashiwo
0,109470,Alexandrium,urn:lsid:marinespecies.org:taxname:109470,Genus,Chromista,Alexandrium_singlet
0,109428,Gonyaulacaceae,urn:lsid:marinespecies.org:taxname:109428,Family,Chromista,Amy_Gony_Protoc


In [27]:
# display the value that has AphiaID as NaN
class_names.loc[51]

Class Name                                                      unclassified
Class ID                                                                  51
Common Name                                                     Unclassified
AphiaID                                                                  NaN
URN                                      urn:lsid:marinespecies.org:taxname:
Grouping                                                               Other
HAB                                                                      NaN
Description                Classification lacked confidence to asssign a ...
Manual Classifier Notes                                                  NaN
Example Images                                                           NaN
Name: 51, dtype: object

In [28]:
# okay we have the class name but not the intended_worms_taxon
# do I need intended_worms_taxon?  We have the AphialID for the class from another file
a2=autoclass
a2=a2.set_index(['pid'],append=False)
a2.index.name=None

In [29]:
a3=a2.stack(future_stack=True)

In [30]:
ik=np.arange(0,len(a3))
for r in ik:
    junk=str(a3[r:r+1]).split()
    if r==0:
        pid=[junk[0]]
        myclass=[junk[1]]
        score=[float(junk[2])]
    else:
        pid.append(junk[0])
        myclass.append(junk[1])
        score.append(float(junk[2]))

In [31]:
bigtable=pd.DataFrame({'pid':pid,'class':myclass,'score':score})

In [32]:
[l1,l2]=threshold.shape

In [33]:
ix=np.arange(0,l1)
zz=0
for kk in ix:
    id1=bigtable['class']==threshold['Class Name'][kk]
    subtable=bigtable[['pid','class','score']][id1]
    id2=subtable['score'] >= threshold['Threshold'][kk]
    smalltable=subtable[['pid','class','score']][id2]
    if smalltable.size > 0:
        if zz==0:
            classtable=smalltable
            zz=zz+1
        else:
            classtable=pd.concat([classtable, smalltable],ignore_index=True)
            #classtable.append(smalltable)

In [34]:
classtable.head(3)

,pid,class,score
0,D20230717T003329_IFCB104_00033,Akashiwo,0.999
1,D20230717T003329_IFCB104_00107,Akashiwo,1.000
2,D20230717T003329_IFCB104_00311,Akashiwo,1.000


In [35]:
# This is an example we want to use later commenting out and cleaning up code
#pidsbyclass=classtable['class']=='Centric'
#mytest=classtable[pidsbyclass]
##mytest['pid']
## print(bin_id)
#rois=mytest['pid'].str.replace(bin_id+'_','')
#rois=rois.astype(int)
##print(rois)

In [36]:
summarytable=classtable.groupby('class').count()
summarytable=summarytable.rename(columns={'score':'occurrences'}).drop('pid',axis=1)

In [37]:
summarytable.head(3)

,occurrences
class,
Akashiwo,7
Alexandrium_singlet,6
Amy_Gony_Protoc,8


In [38]:
# This is important and needs to be addressed before code is complete
# missing occurrence to add 0's
# skip this for now

In [39]:
features

,roi_number,Area,B180,B90,Bflip,Biovolume,BoundingBox_xwidth,BoundingBox_ywidth,ConvexArea,ConvexPerimeter,...,HOG79,HOG80,HOG81,Area_over_PerimeterSquared,Area_over_Perimeter,H90_over_Hflip,H90_over_H180,Hflip_over_H180,summedConvexPerimeter_over_Perimeter,rotated_BoundingBox_solidity
0,2,54574.0,0.368497,0.181079,0.962857,3.758912e+06,575,224,90723.669102,1302.338204,...,0.308797,0.361135,0.316723,0.022128,34.750909,35.205691,2.031675,0.057709,0.829286,0.427089
1,3,1267.0,0.918046,0.825847,0.930654,3.857876e+04,48,36,1327.957732,129.915465,...,0.164193,0.264625,0.485808,0.067791,9.267747,2.203961,2.194031,0.995494,0.950295,0.699945
2,4,1912.0,0.880233,0.866455,0.888182,6.354290e+04,55,52,2247.137266,170.274532,...,0.169124,0.354672,0.445574,0.053102,10.076294,1.283148,1.064763,0.829806,0.897352,0.647341
3,5,7878.0,0.810080,0.622953,0.956329,2.892267e+05,167,95,11551.292600,433.585199,...,0.426714,0.309362,0.332592,0.027104,14.612376,14.979906,2.152259,0.143676,0.804228,0.572041
4,6,3090.0,0.867661,0.787869,0.878365,1.457461e+05,77,63,3718.942534,222.885069,...,0.266490,0.260658,0.400797,0.040571,11.196557,1.960536,1.802546,0.919415,0.807620,0.665013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3005,3007,22180.0,0.631605,0.244364,0.885482,1.064719e+06,425,148,44007.229629,948.459258,...,0.361883,0.362313,0.316244,0.013340,17.201133,4.970794,1.958042,0.393909,0.735553,0.328495
3006,3008,6073.0,0.690331,0.587547,0.705320,1.755594e+05,115,127,10820.651364,394.302728,...,0.146769,0.224298,0.602977,0.021600,11.453202,1.088068,1.021528,0.938845,0.743624,0.417509
3007,3009,6198.0,0.816049,0.554603,0.910856,1.916049e+05,155,115,9919.963414,405.926828,...,0.367091,0.250991,0.289471,0.020879,11.375633,11.090338,3.437931,0.309993,0.745027,0.535430
3008,3010,7818.0,0.784216,0.598491,0.936812,2.823157e+05,179,86,11944.866111,439.732222,...,0.294159,0.545535,0.219234,0.024856,13.940040,11.349296,2.251563,0.198388,0.784073,0.502023


In [40]:
# create event table
# want to create this as a module which we pass information to but for now mostly static...
## event table
# datasetName="santa-cruz-municipal-wharf"
# eventID=bin_id
# eventDate=details['timestamp_iso'] # need to truncate to just the date
# decimalLongitude=details['lng']
# decimalLatitude=details['lat']
# countryCode='US'
# geodeticDatum='WGS84'
# minimumDepthInMeters=
# maximumDepthInMeters=
# sampleSizeValue=details['ml_analyzed']
# sampleSizeUnit='milliliter'
eventdict={'datasetName':'https://ifcb.caloos.org/santa-cruz-municipal-wharf',
           'eventID':bin_id,
           'eventDate':details['timestamp_iso'],
           'institutionCode':'UCSC',
           'decimalLongitude':details['lng'],
           'decimalLatitude':details['lat'],
           'countryCode':'US',
           'geodeticDatum':'WGS84',
           'minimumDepthInMeters':1,
           'maximumDepthInMeters':3,
           'sampleSizeValue':details['ml_analyzed'], # need to remove the ml from the name
           'sampleSizeUnit':'milliliter'}
eventdf=pd.DataFrame.from_dict([eventdict])

In [41]:
eventdf

,datasetName,eventID,eventDate,institutionCode,decimalLongitude,decimalLatitude,countryCode,geodeticDatum,minimumDepthInMeters,maximumDepthInMeters,sampleSizeValue,sampleSizeUnit
0,https://ifcb.caloos.org/santa-cruz-municipal-w...,D20230717T003329_IFCB104,2023-07-17T00:33:29+00:00,UCSC,-122.021868,36.961491,US,WGS84,1,3,3.993 ml,milliliter


In [49]:
# 
# size of summary table
#
ls=len(summarytable)
xworms=wormsdf['Class Name'].reset_index()
# need to find classtable to match class name
# note this doesn't create a table/dataframe but needs to be adjusted as we have one to many needs to fill in columns
#for s in np.arange(0,1):
for s in np.arange(0,ls):
    eventID=details["bin_id"]
    z=xworms['Class Name'][:]==summarytable.index[s]
    zi=np.where(z)
    iz=wormsdf.iloc[zi[0][0]]
    scientificName=iz['scientificname']
    scientificID=iz['lsid']
    taxonRank=iz['rank']
    kingdom=iz['kingdom']
    zc=classtable['class']==summarytable.index[s] # find the class names
    basisOfRecord='MachineObservation'
    occurrenceStatus='Present' # need to deal with adding zeros for absent values but for now this is that case
    verbatimIdentification=classtable['class'][ci[0][0]]
    identifiedBy=''
    identificationVerificationStatus='PredictedByMachine'
    identificationReferences="Trained machine learning model: Daniel, P. (2023-02) phytoClassUCSC - A phytoplankton classifier for IFCB data. Version 1.0. Hugging Face repository. https://huggingface.co/patcdaniel/phytoClassUCSC | Software to run the trained machine learning model: Sosik, H., J. Futrelle, E. Peacock,  T. Golden, J. Lopez (2023-11-13) ifcb-analysis. GitHub repository. https://github.com/tsgolden/ifcb-analysis/commit/9e228c9f616edd85b57aefc0792125ec92f20e31 forked from https://github.com/yosoyjay/ifcb-analysis forked from https://github.com/hsosik/ifcb-analysis | Software to interpret autoclass scores: https://github.com/CeNCOOS/OBIS_workshop_2024_IFCB [Add which script or function] | Input parameters to interpret autoclass scores: https://github.com/CeNCOOS/OBIS_workshop_2024_IFCB/blob/main/class_thresholds_cencoos_classifier_20240412.csv"    
    ci=np.where(zc) # get the index of these name
    lci=len(ci[0])
    # This is not ideal but it works, brute force there must be a better way
    for t in np.arange(0,lci):
        if t==0:
            evid=[]
            evid.append(eventID)
            scnm=[]
            scnm.append(scientificName)
            scid=[]
            scid.append(scientificID)
            txr=[]
            txr.append(taxonRank)
            kg=[]
            kg.append(kingdom)
            br=[]
            br.append(basisOfRecord)
            ocs=[]
            ocs.append(occurrenceStatus)
            vbi=[]
            vbi.append(verbatimIdentification)
            ibi=[]
            ibi.append('')
            ivs=[]
            ivs.append(identificationVerificationStatus)
            idr=[]
            idr.append(identificationReferences)
        else:
            evid.append(eventID)
            scnm.append(scientificName)
            scid.append(scientificID)
            txr.append(taxonRank)
            kg.append(kingdom)
            br.append(basisOfRecord)
            ocs.append(occurrenceStatus)
            vbi.append(verbatimIdentification)
            ibi.append('')
            ivs.append(identificationVerificationStatus)
            idr.append(identificationReferences)           
    occurrenceID=classtable['pid'][ci[0][:]]
    tmpdict={'eventID':evid,'occurrenceID':occurrenceID,'scientificName':scnm,'scienfificID':scid,'taxonRank':txr,
            'kingdom':kg,'basiOfRecord':br,'OccurrenceStatus':ocs,'verbatimIdentification':vbi,'identifiedBy':ibi,'identificationVerificationStatus':ivs,
            'identificationReferences':idr}
    occframe=pd.DataFrame.from_dict(tmpdict)
    if s==0:
        occurrencedf=occframe
    else:
        occurrencedf=pd.concat([occurrencedf,occframe])    
# see about table elements occurrence table
#eventID=details["bin_id"]
#occurrenceID=classtable["pid"]
#basisOfRecord="MachineObservation"
#identifiedBy=""
#identificationVerificationStatus="PredictedByMachine"
#identificationReferences="Trained machine learning model: Daniel, P. (2023-02) phytoClassUCSC - A phytoplankton classifier for IFCB data. Version 1.0. Hugging Face repository. https://huggingface.co/patcdaniel/phytoClassUCSC | Software to run the trained machine learning model: Sosik, H., J. Futrelle, E. Peacock,  T. Golden, J. Lopez (2023-11-13) ifcb-analysis. GitHub repository. https://github.com/tsgolden/ifcb-analysis/commit/9e228c9f616edd85b57aefc0792125ec92f20e31 forked from https://github.com/yosoyjay/ifcb-analysis forked from https://github.com/hsosik/ifcb-analysis | Software to interpret autoclass scores: https://github.com/CeNCOOS/OBIS_workshop_2024_IFCB [Add which script or function] | Input parameters to interpret autoclass scores: https://github.com/CeNCOOS/OBIS_workshop_2024_IFCB/blob/main/class_thresholds_cencoos_classifier_20240412.csv"
#identificationRemarks="Arbitrary threshold used for both presence and absence without testing for false positives."
#associatedMedia='https://ifcb.caloos.org/image?dataset=santa-cruz-municipal-wharf&bin=bin_id&image=XXXX
#verbatimIdentification
#scientificName=wormsdf['scientificname'] # need to match up with
#scientificID=wormsdf['lsid']
#taxonRank=wormsdf['rank']
#kingdom=wormdf['kingdom'] # but needs to be matched to id
#occurrenceStatus present/absent
#organismQuantity=summarytable['occurrences']./mlanalyzed
#organismQuantityType='counts per milliliter'
#institutionCode

In [50]:
occurrencedf.head(3)

,eventID,occurrenceID,scientificName,scienfificID,taxonRank,kingdom,basiOfRecord,OccurrenceStatus,verbatimIdentification,identifiedBy,identificationVerificationStatus,identificationReferences
0,D20230717T003329_IFCB104,D20230717T003329_IFCB104_00033,Akashiwo sanguinea,urn:lsid:marinespecies.org:taxname:232546,Species,Chromista,MachineObservation,Present,Akashiwo,,PredictedByMachine,"Trained machine learning model: Daniel, P. (20..."
1,D20230717T003329_IFCB104,D20230717T003329_IFCB104_00107,Akashiwo sanguinea,urn:lsid:marinespecies.org:taxname:232546,Species,Chromista,MachineObservation,Present,Akashiwo,,PredictedByMachine,"Trained machine learning model: Daniel, P. (20..."
2,D20230717T003329_IFCB104,D20230717T003329_IFCB104_00311,Akashiwo sanguinea,urn:lsid:marinespecies.org:taxname:232546,Species,Chromista,MachineObservation,Present,Akashiwo,,PredictedByMachine,"Trained machine learning model: Daniel, P. (20..."


In [ ]:
# EMoF table
# eventID
# eventID=details['bin_id']
# measurementType='Sampling Instrument Name'
# measurementTypeID='http://vocab.nerc.ac.uk/collection/Q01/current/Q0100002/'
# measurementValue='McLane Research Laboratories Imaging FlowCytobot imaging sensor'
# measurementValueID='http://vocab.nerc.ac.uk/collection/L22/current/TOOL1588/'
# measurementType='Sample Volume'
# measurementTypeID='http://vocab.nerc.ac.uk/collection/P01/current/VOLXXXXX/'
# measurementValue=analyzed_volume
# measurementUnit='Millilitres'
# measurementUnitID='http://vocab.nerc.ac.uk/collection/P06/current/VVML/'
# occurrenceID
# measurementRemarks='abundance per milliliter, inclusive of single cells and chains'
# measurementValueID
# measurementUnit
# measurementUnitID
# measurementID
#           'sampleSizeValue':details['ml_analyzed'], # need to remove the ml from the name
#           'sampleSizeUnit':'milliliter'}
